In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
#pip install spectrum_utils

In [ ]:
#pip install pyteomics

In [4]:
#!pip install rdkit

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs

In [6]:
import logging
from typing import Dict, IO, Iterator, Sequence, Union
from pyteomics import mgf
import pyteomics
from spectrum_utils.spectrum import MsmsSpectrum
import matplotlib.pyplot as plt
import spectrum_utils.plot as sup
import spectrum_utils as su
import numpy as np
from src.load_data import LoadData
from spectrum_utils import plot
from src.preprocessor import Preprocessor
from src.ml_model import MlModel
from src.tanimoto import Tanimoto
from sklearn.model_selection import train_test_split
from src.plotting import Plotting
import pickle
from src.cosine_distance import CosineDistance

In [7]:
from tqdm import tqdm

In [8]:
from src.spectrum_ext import SpectrumExt

In [9]:
from src.train_utils import TrainUtils
from sklearn.metrics import r2_score

In [10]:
pp =Preprocessor()

In [11]:
## THINGS TO DO:
# ADD ADITIONAL VARIABLES TO VECTOR
# TRAIN FOR MORE TIME

## Parameters

In [12]:
mgf_path = r"C:\Users\ramir\Downloads\ALL_GNPS_NO_PROPOGATED.mgf"

## Load data + preprocessing

In [13]:
all_spectrums_original = LoadData.get_all_spectrums(mgf_path,1000000)

 50%|█████     | 503801/1000000 [03:53<03:49, 2160.76it/s] 


We reached the end of the array at index 503801


In [14]:
len(all_spectrums_original)

503801

In [15]:
#preprocess 
all_spectrums = pp.preprocess_all_spectrums(all_spectrums_original)

503801it [00:26, 19319.50it/s]
100020it [00:06, 16617.28it/s]


In [16]:
len(all_spectrums)

100020

## Divide data into training/test  + computing similarity score

In [17]:
# divide train and test
all_spectrums_train, all_spectrums_test = train_test_split(all_spectrums)

In [ ]:
molecule_pairs_train= TrainUtils.compute_all_tanimoto_results(all_spectrums_train, max_combinations=10**7)

 92%|█████████▏| 9249227/10000000 [1:24:58<06:44, 1856.41it/s]

In [ ]:
molecule_pairs_test = TrainUtils.compute_all_tanimoto_results(all_spectrums_test, max_combinations=10**5)

## Save data

In [ ]:
import pickle
# Dump the dictionary to a file using pickle

dataset ={'all_spectrums_train':all_spectrums_train,
          'all_spectrums_test': all_spectrums_test,
          'molecule_pairs_train':molecule_pairs_train,
          'molecule_pairs_test': molecule_pairs_test,
         }
with open('./dataset_processed.pkl', 'wb') as file:
    pickle.dump(dataset, file)

## Load data

In [ ]:
#import pickle

# Load the dataset from the pickle file
#with open('./dataset_processed.pkl', 'rb') as file:
#    dataset = pickle.load(file)

# Now you can access the original variables from the dataset dictionary
#all_spectrums_train = dataset['all_spectrums_train']
#all_spectrums_test = dataset['all_spectrums_test']
#molecule_pairs_train = dataset['molecule_pairs_train']
#molecule_pairs_test = dataset['molecule_pairs_test']


## Balance the data

In [ ]:
plt.hist([m.similarity for m in molecule_pairs_train], bins=3)

In [ ]:
# get a uniform distribution of taminoto scores
uniformed_molecule_pairs_train =TrainUtils.uniformise(molecule_pairs_train)

In [ ]:
_=plt.hist([m.similarity for m in uniformed_molecule_pairs_train], bins=3)

## Train Siamese model

In [ ]:
model = MlModel(input_dim=all_spectrums_train[0].spectrum_vector.shape[0])

In [ ]:
model.compile()

In [ ]:
model.fit(uniformed_molecule_pairs_train, epochs=100)

In [ ]:
model.load_best_model()



## Evaluate performance

In [ ]:
prediction_results_train = model.predict(uniformed_molecule_pairs_train)

In [ ]:
plt.hist(prediction_results_train, bins=10)

In [ ]:
# get a uniform distribution of taminoto scores
uniformed_molecule_pairs_test =TrainUtils.uniformise(molecule_pairs_test)

In [ ]:
prediction_results_test = model.predict(uniformed_molecule_pairs_test)

In [ ]:

plt.scatter([m.similarity for m in uniformed_molecule_pairs_train], prediction_results_train, alpha=0.01)
plt.title('Prediction of Taminoto score (TRAIN)')
plt.xlabel('true')
plt.ylabel('pred')
plt.grid()

In [ ]:
plt.plot([m.similarity for m in uniformed_molecule_pairs_test])

In [ ]:
plt.plot(prediction_results_test)

In [ ]:
plt.scatter([m.similarity for m in uniformed_molecule_pairs_test], prediction_results_test, alpha=1)
plt.title('Prediction of Taminoto score (test)')
plt.xlabel('true')
plt.ylabel('pred')
plt.grid()

In [ ]:
r2_score([m.similarity for m in uniformed_molecule_pairs_test], prediction_results_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
mean_squared_error(new_labels, prediction_results_train)

In [ ]:
mean_squared_error(labels_test_uniform, y_pred)

In [ ]:
plt.hist(labels_test_uniform)

In [ ]:
plt.hist(y_pred)
plt.grid()
plt.title('Histogram predictions')
plt.xlabel('prediction')

# equal molecules

In [ ]:
high_molecules= [m for m in uniformed_molecule_pairs_train if m.similarity==1]

In [ ]:
index=11

In [ ]:
for s in [high_molecules[index].spectrum_object_0, high_molecules[index].spectrum_object_1]:
    Plotting.plot_spectrum(s)

In [ ]:
high_molecules[index].params_0

In [ ]:
high_molecules[index].params_1

In [ ]:
Plotting.plot_molecule_pair_spectrum(high_molecules[index])

In [ ]:
CosineDistance.compute_cosine_distance(high_molecules[index].vector_0, high_molecules[index].vector_1)

In [ ]:
plt.hist([CosineDistance.compute_cosine_distance(p.vector_0, p.vector_1) for p in high_molecules])

## Low molecules

In [ ]:
low_molecules= [m for m in uniformed_molecule_pairs_train if m.similarity<0.1]

In [ ]:
index_low=2

In [ ]:
Plotting.plot_molecule_pair_spectrum(low_molecules[index_low])


In [ ]:
CosineDistance.compute_cosine_distance(low_molecules[index_low].vector_0, low_molecules[index_low].vector_1)

In [ ]:
plt.hist([CosineDistance.compute_cosine_distance(p.vector_0, p.vector_1) for p in low_molecules])

In [ ]:
-